In [ ]:
# @title Imports, initial setup (Ctrl+F9 to run all)
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt

try:
    import gamry_parser
except:
    subprocess.run(["pip", "install", "gamry-parser"], encoding="utf-8", shell=False)
finally:
    import gamry_parser

p = parser.CyclicVoltammetry()

print("Done.")

In [ ]:
"""
### SCRIPT CONFIGURATION SETTINGS ###
"""

"""
DATA SOURCE
"""
upload_files = True

"""
PLOTTING
Plots are generated in the notebook. They are not saved / exported.
"""
show_plots = True  # do we want to show analysis plots in this notebook?
compare_curves = 3  # compare a specific curve across files


print("Done.")

In [ ]:
#@title DTA File
if upload_files:
  experiment = files.upload()
else:
  !wget -c https://raw.githubusercontent.com/bcliang/gamry-parser/master/tests/cv_data.dta
  experiment = ["cv_data.dta"]



In [ ]:
# @title Load and Plot Curve Data

for f in experiment:
    p.load(f)

    # generate a plot based on the first curve listed in the file.
    data = p.curve(p.curve_count)

    # print to screen
    print("Data Preview: Last Curve")
    print(data.iloc[:5])

    # matplotlib fig
    if show_plots:
        fig, ax = plt.subplots(figsize=(18, 8))
        for i in range(p.curve_count):
            data = p.curve(i)
            trace = ax.plot(data["Vf"], data["Im"] * 1e6, label="curve {}".format(i))

        ax.set_title("{}, {} curves".format(f, p.curve_count), fontsize=18)
        ax.set_xlabel("Potential (V)")
        ax.set_ylabel("Current (A)", fontsize=14)
        plt.show()

In [ ]:
# @title Compare a specific curve across loaded files

fig, ax = plt.subplots(figsize=(18, 8))

for f in experiment:
    p.load(f)

    # generate a plot based on the first curve listed in the file.
    if p.curve_count > compare_curves:
        data = p.curve(compare_curves)
        trace = ax.plot(data["Vf"], data["Im"] * 1e6, label="file {}".format(f))

ax.set_title(
    "CyclicVoltammetry Test, Compare Curve #{}".format(compare_curves), fontsize=18
)
ax.set_xlabel("Potential (V)")
ax.set_ylabel("Current (A)", fontsize=14)
ax.legend()
plt.show()

In [ ]:
# @title Download All Curves, All Experiments

aggreg = pd.DataFrame()

for f in experiment:
    p.load(f)

    # use the curves @property to retrieve all curve data
    for df in p.curves:
        aggreg = aggreg.append(df)


aggreg.to_csv("results.csv")
files.download("results.csv")